In [2]:
# Inport Required Libraries
import tensorflow as tf
import sys
import numpy as np
import glob
import os
import re
import shutil
from google.colab import files
!pip install ffmpeg-python &> 0
!apt-get update && apt-get -qq install xxd

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:15 http://ppa

In [ ]:
# We Download the Tensorflow's Speech Command Dataset as the foundation to our dataset.
!wget https://storage.googleapis.com/download.tensorflow.org/data/speech_commands_v0.02.tar.gz
DATASET_DIR =  'dataset/'
!mkdir dataset
!tar -xf speech_commands_v0.02.tar.gz -C 'dataset'
!rm -r -f speech_commands_v0.02.tar.gz

--2021-03-21 21:26:00--  https://storage.googleapis.com/download.tensorflow.org/data/speech_commands_v0.02.tar.gz
Resolving storage.googleapis.com (storage.googleapis.com)... 172.253.119.128, 108.177.111.128, 142.250.103.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.253.119.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2428923189 (2.3G) [application/gzip]
Saving to: ‘speech_commands_v0.02.tar.gz’

speech_commands_v0. 100%[===================>]   2.26G  52.8MB/s    in 19s     

2021-03-21 21:26:19 (122 MB/s) - ‘speech_commands_v0.02.tar.gz’ saved [2428923189/2428923189]



In [ ]:
uploaded = files.upload()

Saving Fan_1616125977673(1).ogg to Fan_1616125977673(1).ogg
Saving Fan_1616125977673(2).ogg to Fan_1616125977673(2).ogg
Saving Fan_1616125977673(3).ogg to Fan_1616125977673(3).ogg
Saving Fan_1616125977674.ogg to Fan_1616125977674.ogg
Saving Fan_1616125977674(1).ogg to Fan_1616125977674(1).ogg
Saving Fan_1616125977674(2).ogg to Fan_1616125977674(2).ogg
Saving Fan_1616125977674(3).ogg to Fan_1616125977674(3).ogg
Saving Fan_1616125977674(4).ogg to Fan_1616125977674(4).ogg
Saving Fan_1616125977675.ogg to Fan_1616125977675.ogg
Saving Fan_1616125977675(1).ogg to Fan_1616125977675(1).ogg
Saving Fan_1616125977675(2).ogg to Fan_1616125977675(2).ogg
Saving Fan_1616125977676.ogg to Fan_1616125977676.ogg
Saving Fan_1616125977676(1).ogg to Fan_1616125977676(1).ogg
Saving Fan_1616125977676(2).ogg to Fan_1616125977676(2).ogg
Saving Fan_1616125977677.ogg to Fan_1616125977677.ogg
Saving Fan_1616125977677(1).ogg to Fan_1616125977677(1).ogg
Saving Fan_1616125977677(2).ogg to Fan_1616125977677(2).ogg
Savi

In [4]:
# convert the ogg files to wavs
!mkdir wavs
!find *.ogg -print0 | xargs -0 basename -s .ogg | xargs -I {} ffmpeg -i {}.ogg -ar 16000 wavs/{}.wav
!rm -r -f *.ogg

# we then use pete's tool to only extract 1 second clips from them for use with the KWS pipeline
!mkdir trimmed_wavs
!git clone https://github.com/petewarden/extract_loudest_section.git
!make -C extract_loudest_section/
!/tmp/extract_loudest_section/gen/bin/extract_loudest_section 'wavs/*.wav' trimmed_wavs/
!rm -r -f /wavs

Streaming output truncated to the last 5000 lines.
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-libzmq --enable-libzvbi --enab

In [5]:
# Store them in the appropriate folders
data_index = {}
os.chdir('trimmed_wavs')
search_path = os.path.join('*.wav')
for wav_path in glob.glob(search_path):
    matches = re.search('([^/_]+)_([^/_]+)\.wav', wav_path)
    word = matches.group(1).lower()
    instance = matches.group(2).lower()
    if not word in data_index:
      data_index[word] = {}
    data_index[word][instance] = wav_path

output_dir = os.path.join('..', 'dataset')
try:
    os.mkdir(output_dir)
except:
    pass
for word in data_index:
  word_dir = os.path.join(output_dir, word)
  try:
      os.mkdir(word_dir)
      print('Created dir: ' + word_dir)
  except:
      print('Storing in existing dir: ' + word_dir)
  for instance in data_index[word]:
    wav_path = data_index[word][instance]
    output_path = os.path.join(word_dir, instance + '.wav')
    shutil.copyfile(wav_path, output_path)
os.chdir('..')
!rm -r -f trimmed_wavs

Created dir: ../dataset/light
Created dir: ../dataset/fan
Created dir: ../dataset/heater


In [ ]:
!zip -r speechCommand.zip dataset

Streaming output truncated to the last 5000 lines.
  adding: dataset/happy/37bd115d_nohash_0.wav (deflated 22%)
  adding: dataset/happy/7c9b43f0_nohash_0.wav (deflated 42%)
  adding: dataset/happy/da584bc0_nohash_0.wav (deflated 27%)
  adding: dataset/happy/8e884ec4_nohash_0.wav (deflated 38%)
  adding: dataset/happy/3ed3e49a_nohash_1.wav (deflated 37%)
  adding: dataset/happy/65d81c54_nohash_0.wav (deflated 30%)
  adding: dataset/happy/31601aad_nohash_0.wav (deflated 26%)
  adding: dataset/happy/ab9b93e4_nohash_0.wav (deflated 52%)
  adding: dataset/happy/531a5b8a_nohash_0.wav (deflated 43%)
  adding: dataset/happy/f4c77b26_nohash_0.wav (deflated 34%)
  adding: dataset/happy/8d4cdc60_nohash_0.wav (deflated 8%)
  adding: dataset/happy/113b3fbc_nohash_0.wav (deflated 21%)
  adding: dataset/happy/19f9c115_nohash_0.wav (deflated 24%)
  adding: dataset/happy/c0e8f5a1_nohash_0.wav (deflated 16%)
  adding: dataset/happy/f5e44d34_nohash_0.wav (deflated 35%)
  adding: dataset/happy/093f65a1_no

In [8]:
!zip -r dataset.zip dataset

updating: dataset/ (stored 0%)
  adding: dataset/fan/ (stored 0%)
  adding: dataset/fan/1616758198886.wav (deflated 54%)
  adding: dataset/fan/1616758282603.wav (deflated 14%)
  adding: dataset/fan/1616758118151.wav (deflated 49%)
  adding: dataset/fan/1616758198889.wav (deflated 47%)
  adding: dataset/fan/1616758198891.wav (deflated 48%)
  adding: dataset/fan/1616758078260.wav (deflated 13%)
  adding: dataset/fan/1616758017771.wav (deflated 11%)
  adding: dataset/fan/1616758118163.wav (deflated 36%)
  adding: dataset/fan/1616758365125.wav (deflated 14%)
  adding: dataset/fan/1616758118162.wav (deflated 46%)
  adding: dataset/fan/1616758365124.wav (deflated 10%)
  adding: dataset/fan/1616758403179.wav (deflated 10%)
  adding: dataset/fan/1616758158349.wav (deflated 54%)
  adding: dataset/fan/1616758403176.wav (deflated 17%)
  adding: dataset/fan/1616758078259.wav (deflated 10%)
  adding: dataset/fan/1616758403173.wav (deflated 15%)
  adding: dataset/fan/1616757980213.wav (deflated 14%)